In [ ]:
!pip install -U transformers
!pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer, default_data_collator,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import sys
sys.path.append('../../src')
from min_preprocessing import FinancialTweetPreprocessor
from evaluation_transformer import evaluate_transformer
from initial_balanced_dataset import create_balanced_dataset

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Create dataset (skips if dataset already exists)
create_balanced_dataset()

# Load Data
df = pd.read_csv('../../dataset/initial_balanced_tweets.csv')
print(df.shape)

# Preprocess
print("Preprocessing DataFrame...")
preprocessor = FinancialTweetPreprocessor()
df_preprocessed = preprocessor.preprocess_dataset(df, 'tweet')

pd.set_option('display.max_columns', None)
print("\nProcessed DataFrame:")
print(df_preprocessed[['processed_text', 'sentiment', 'ticker_count', 'mention_count',
                       'url_count', 'token_count', 'exclamation_count', 'question_count']].head())
print("\nAll columns in processed DataFrame:")
print(df_preprocessed.columns.tolist())

# Count zeros in every column of df_preprocessed
zero_counts = (df_preprocessed == 0).sum()
print("Number of zeros per column:")
print(zero_counts)

(47106, 2)
Preprocessing DataFrame...

Processed DataFrame:
                                      processed_text  sentiment  ticker_count  \
0  upholding and perhaps pushing price upwards here.          1             0   
1  Michael K. Wirth Sells 52,500 Shares of Chevro...          2             1   
2  Why would you buy AAP when you can buy CS wher...          1             0   
3  Economic experts believe that the current inte...          0             0   
4          The era of financial boom is over forever          2             0   

   mention_count  url_count  token_count  exclamation_count  question_count  
0              0          0            6                  0               0  
1              0          0           13                  0               0  
2              0          0           15                  0               0  
3              0          0           10                  0               0  
4              0          0            5                  0    

In [ ]:
X = df_preprocessed[['processed_text', 'ticker_count', 'mention_count',
                 'url_count', 'token_count']]
y = df_preprocessed['sentiment']

# Split the Data into Training and Testing Sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Training set size: {len(X_train)} samples")
print(f"Validation set size: {len(X_val)} samples")

Training set size: 37684 samples
Validation set size: 9422 samples


In [ ]:
# ------ FinBert + LoRA ------
print(f"\nDataset Configuration")
print(f"\nClass distribution in training set:")
print(y_train.value_counts().sort_index())
print(f"\nClass distribution in validation set:")
print(y_val.value_counts().sort_index())


Dataset Configuration

Class distribution in training set:
sentiment
0    12562
1    12561
2    12561
Name: count, dtype: int64

Class distribution in validation set:
sentiment
0    3140
1    3141
2    3141
Name: count, dtype: int64


In [ ]:
print("\nLOADING & PREPROCESSING TEST SET")
df_test = pd.read_csv("../../dataset/test_set.csv")
# Preprocess text
preprocessor = FinancialTweetPreprocessor()
df_test = preprocessor.preprocess_dataset(df_test, text_column='tweet')
# Prepare inputs
X_test = df_test[['processed_text', 'ticker_count', 'mention_count', 'url_count', 'token_count']]
y_test = df_test['sentiment']
print(f"Testing set size: {len(X_test)} samples")

In [ ]:
print("\nLOADING FINBERT AND CONFIGURING LORA")

# Load FinBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

# Load the base model with the quantization config
base_model = AutoModelForSequenceClassification.from_pretrained(
    "ProsusAI/finbert",
    num_labels=3,
)
base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=16,  # rank
    lora_alpha=32,
    target_modules=["query", "key", "value", "dense"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
)

# Apply LoRA to the base model
lora_model = get_peft_model(base_model, lora_config)
print("\nLoRA configuration applied!")
lora_model.print_trainable_parameters()


LOADING FINBERT AND CONFIGURING QLORA


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]


LoRA configuration applied!
trainable params: 2,681,091 || all params: 112,165,638 || trainable%: 2.3903


In [ ]:
# Verify Model Architecture
print("PeftModel dirs:", dir(lora_model))
print("Does it have .base_model? ", hasattr(lora_model, "base_model"))

print("base_model dirs:", dir(lora_model.base_model))

print("Does base_model have .model? ", hasattr(lora_model.base_model, "model"))

if hasattr(lora_model, "get_base_model"):
    print("Using get_base_model():", dir(lora_model.get_base_model()))
    print("Has .bert on get_base_model()? ", hasattr(lora_model.get_base_model(), "bert"))
else:
    print("Has .bert on base_model? ", hasattr(lora_model.base_model, "bert"))

PeftModel dirs:

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

 ['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_check_new_adapter_config', '_compiled_call_impl', '_create_repo', '_enable_peft_forward_hooks', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_base_model_class', '_get_files_timestamps', '_get_name', '_is_full_backward_hook', '_is_prompt_learning', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks',

In [ ]:
print("\nCREATING CUSTOM MODEL WITH METADATA FEATURES")
class FinBERTWithMetadata(nn.Module):
    def __init__(self, lora_model, num_features=4, num_labels=3):
        super().__init__()
        self.lora_model = lora_model
        self.dropout = nn.Dropout(0.1)
        # Combine BERT hidden size (768) with metadata features
        self.classifier = nn.Linear(768 + num_features, num_labels)

    def forward(self, input_ids, attention_mask, features=None):
        # Get BERT outputs
        bert_outputs = self.lora_model.base_model.model.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # Get pooled output
        pooled_output = bert_outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        # If features provided, concatenate them
        if features is not None:
            combined = torch.cat([pooled_output, features], dim=1)
            logits = self.classifier(combined)
        else:
            # Fallback for inference without features
            logits = self.classifier(pooled_output)

        return logits

# Initialize the combined model
model = FinBERTWithMetadata(lora_model)
print("Custom model with metadata features created!")


CREATING CUSTOM MODEL WITH METADATA FEATURES
Custom model with metadata features created!


In [ ]:
print("\nPREPARING DATA WITH TEXT AND METADATA")

# Scale metadata features
scaler = StandardScaler()
feature_columns = ['ticker_count', 'mention_count', 'url_count', 'token_count']
train_features = X_train[feature_columns].values
val_features = X_val[feature_columns].values
test_features = X_test[feature_columns].values

print("Scaling metadata features...")
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

print(f"Feature statistics (after scaling):")
print(f"Mean: {train_features_scaled.mean(axis=0)}")
print(f"Std: {train_features_scaled.std(axis=0)}")

# Tokenize text
def tokenize_function(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128
    )


print("\nTokenizing text data...")
train_encodings = tokenize_function(X_train['processed_text'].tolist())
val_encodings = tokenize_function(X_val['processed_text'].tolist())
test_encodings = tokenize_function(X_test['processed_text'].tolist())

# Custom dataset class that includes features
class TextAndFeaturesDataset(Dataset):
    def __init__(self, encodings, labels, features):
        self.encodings = encodings
        self.labels = labels
        self.features = features

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        item['features'] = torch.tensor(self.features[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextAndFeaturesDataset(train_encodings, y_train.tolist(), train_features_scaled)
val_dataset = TextAndFeaturesDataset(val_encodings, y_val.tolist(), val_features_scaled)
test_dataset = TextAndFeaturesDataset(test_encodings, y_test.tolist(), test_features_scaled)

print(f"Training set size: {len(train_dataset)} samples")
print(f"Validation set size: {len(val_dataset)} samples")
print(f"Test set size: {len(test_dataset)} samples")


PREPARING DATA WITH TEXT AND METADATA
Scaling metadata features...
Feature statistics (after scaling):
Mean: [-3.31853098e-17  2.16835831e-17 -5.07207292e-17 -3.16768866e-17]
Std: [1. 1. 1. 1.]

Tokenizing text data...


In [ ]:
print("SETTING UP CUSTOM TRAINER")

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        features = inputs.pop("features")

        # Forward pass with features
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            features=features
        )

        # Compute loss
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.view(-1, 3), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        labels = inputs.pop("labels") if "labels" in inputs else None
        features = inputs.pop("features")

        with torch.no_grad():
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                features=features
            )

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(outputs.view(-1, 3), labels.view(-1))

        return (loss, outputs, labels)

SETTING UP CUSTOM TRAINER


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="../../models/v1-1/baseline/finbert_lora_min_preproc",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    learning_rate=2e-5,
    fp16=torch.cuda.is_available(),
    seed=42,
    logging_strategy="epoch",
    eval_strategy="epoch",
    logging_dir="../../training_logs/v1-1/baseline/finbert_lora_min_preproc_logs",
    save_strategy="epoch",
    disable_tqdm=True,
    report_to="none",
    remove_unused_columns=False,
)

# Create trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

print("TRAINING MODEL")
trainer.train()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels', 'features'])
{'input_ids': tensor([ 101, 3795, 3006, 4106, 6083, 6706, 3930, 1999, 1996, 2379, 2925, 1012,
         102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0

TrainOutput(global_step=11780, training_loss=0.4736825784317302, metrics={'train_runtime': 3072.8415, 'train_samples_per_second': 61.318, 'train_steps_per_second': 3.834, 'train_loss': 0.4736825784317302, 'epoch': 5.0})

In [ ]:
# Evaluate
print("EVALUATE ON TEST SET")
evaluate_transformer(trainer, train_dataset, test_dataset, y_train, y_test, model, save_dir="../../evaluation/baseline/finbert_lora_min_preproc_eval")

EVALUATE ON TEST SET
EVALUATING ON TEST SET
Making predictions on test set...
Making predictions on training set for comparison...

--- TEST SET PERFORMANCE ---
                Precision (Macro Avg)  Recall (Macro Avg)  \
FinBERT + LoRA               0.772115            0.768307   

                F1-Score (Macro Avg)  Overall Accuracy  
FinBERT + LoRA              0.768498          0.768307  

Detailed Classification Report (Test Set):
              precision    recall  f1-score   support

 Neutral (0)       0.78      0.79      0.78      1666
 Bullish (1)       0.72      0.79      0.75      1666
 Bearish (2)       0.82      0.72      0.77      1666

    accuracy                           0.77      4998
   macro avg       0.77      0.77      0.77      4998
weighted avg       0.77      0.77      0.77      4998


--- TRAINING SET PERFORMANCE ---
Training Accuracy: 0.8648
Test Accuracy: 0.7683
Difference (Train - Test): 0.0965

--- CONFUSION MATRIX (TEST SET) ---

--- TRAINING HISTORY --

In [ ]:
print("SAVING FINAL MODEL AND COMPONENTS")

save_dir = "../../models/v1-1/baseline/finbert_lora_min_preproc/final"
os.makedirs(save_dir, exist_ok=True)

# Save model state (custom classifier + LoRA)
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'feature_columns': feature_columns,
    'lora_config': lora_config,
}, os.path.join(save_dir, 'model_complete.pth'))

# Save tokenizer and LoRA adapter separately
tokenizer.save_pretrained(save_dir)
model.lora_model.save_pretrained(os.path.join(save_dir, 'lora_adapter'))

print(f"Saved to: {save_dir}")
print("Contents:")
print("  - model_complete.pth")
print("  - tokenizer/")
print("  - lora_adapter/")


SAVING FINAL MODEL AND COMPONENTS
Saved to: /content/drive/MyDrive/FNLP/models/v1-1/baseline/finbert_qlora_min_preproc/final
Contents:
  - model_complete.pth
  - tokenizer/
  - lora_adapter/
